In [1]:
import requests, zipfile, io
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image
from io import BytesIO
import random
import tensorflow as tf
import time
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
import requests
labels = 'https://bean-team-sagemaker.s3.us-west-2.amazonaws.com/labels+and+CUI+counts/PADCHEST_chest_x_ray_images_labels_160K_01.02.19.csv'
print('done')


done


/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p27/lib/python2.7/site-packages/OpenSSL/crypto.py:12: CryptographyDeprecationWarning: Python 2 is no longer supported by the Python core team. Support for it is now deprecated in cryptography, and will be removed in the next release.
  from cryptography import x509


In [2]:
!pip install --upgrade pip
!pip install ipykernel --upgrade
!python3 -m ipykernel install --user

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
Requirement already up-to-date: pip in /home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p27/lib/python2.7/site-packages (20.3.4)
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
     |████████████████████████████████| 109 kB 6.6 MB/s eta 0:00:01
  Attempting uninstall: ipy

In [3]:
import boto3

s3_client = boto3.client('s3')
s3_client.download_file('bean-team-sagemaker','labels and CUI counts/PADCHEST_chest_x_ray_images_labels_160K_01.02.19.csv', 'labels.csv')

/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p27/lib/python2.7/site-packages/boto3/compat.py:86: PythonDeprecationWarning: Boto3 will no longer support Python 2.7 starting July 15, 2021. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.6 or later. More information can be found here: https://aws.amazon.com/blogs/developer/announcing-end-of-support-for-python-2-7-in-aws-sdk-for-python-and-aws-cli-v1/
  warnings.warn(warning, PythonDeprecationWarning)


In [4]:
lbls = pd.read_csv('labels.csv',low_memory = False)

lbls0 = lbls[lbls['ImageDir'] ==0]
lbls0 = lbls0.append(lbls[lbls['ImageDir'] ==1])
lbls0 = lbls0.append(lbls[lbls['ImageDir'] ==2])

ours = lbls0[['ImageID','ImageDir','PatientBirth','PatientSex_DICOM','Labels']]

In [5]:
#we have some 9000 pics, trying to load 4200
t1 = time.time()
ct =0
s3_client = boto3.client('s3')
starts = [600,1200,1800]
for j in range(len(starts)):
    M = []
    diagnoses = []
    for i in range(starts[j],starts[j]+600):
        aws_loc = str(ours['ImageDir'][i])+'/'+ours['ImageID'][i]
        loc = 'pics/'+str(ours['ImageDir'][i])+'_'+ours['ImageID'][i]
        s3_client.download_file('bean-team-sagemaker', aws_loc, loc)
        #we downloaded the image
        diagnoses.append(ours['Labels'][i][1:-1])
        #labels added
        ima = Image.open(loc)
        ima = ima.resize((1800,1800))
        arr = np.asarray(ima).astype('int16')
        mean,std = np.mean(arr),np.std(arr)
        if std!=0:
            #normalizing
            arr = ((arr-mean)/std).astype('float16')
            #saving the image at set resolution
            M+=[arr]
            dt = time.time()-t1
            ct+=1
            if ct%100==0:
                print('seg',j,np.round(dt), 'secs',ct,'done')
        os.remove(loc)
        if i%300==0:
            Xall = np.asarray(M).astype('float16')
            np.save('xAll'+str(j),Xall)
            np.save('diagnosesAll'+str(j),diagnoses)

    Xall = np.asarray(M).astype('float16')
    np.save('xAll'+str(j),Xall)
    np.save('diagnosesAll'+str(j),diagnoses)

('seg', 0, 49.0, 'secs', 100, 'done')
('seg', 0, 98.0, 'secs', 200, 'done')
('seg', 0, 150.0, 'secs', 300, 'done')
('seg', 0, 210.0, 'secs', 400, 'done')
('seg', 0, 256.0, 'secs', 500, 'done')
('seg', 0, 304.0, 'secs', 600, 'done')
('seg', 1, 369.0, 'secs', 700, 'done')
('seg', 1, 418.0, 'secs', 800, 'done')
('seg', 1, 467.0, 'secs', 900, 'done')
('seg', 1, 523.0, 'secs', 1000, 'done')
('seg', 1, 571.0, 'secs', 1100, 'done')
('seg', 1, 617.0, 'secs', 1200, 'done')
('seg', 2, 683.0, 'secs', 1300, 'done')
('seg', 2, 733.0, 'secs', 1400, 'done')
('seg', 2, 781.0, 'secs', 1500, 'done')
('seg', 2, 836.0, 'secs', 1600, 'done')
('seg', 2, 883.0, 'secs', 1700, 'done')
